# Setup Environment for Lakeflow Declarative Pipeline Exercise

This notebook sets up the environment for the Lakeflow Declarative Pipeline exercise.

It will:
- Create catalog: `ldp_exercise`
- Create schema: `exercise_schema`
- Create volume: `raw` within the schema
- Create folders (orders, customers, products, payments) in the volume
- Generate sample data files with **intentional data quality issues** for students to resolve

## Step 1: Define Environment Variables

In [0]:
# Define catalog, schema, and volume names
CATALOG_NAME = 'ldp_exercise'
SCHEMA_NAME = 'exercise_schema'
VOLUME_NAME = 'raw'

# Define the base volume path
VOLUME_PATH = f'/Volumes/{CATALOG_NAME}/{SCHEMA_NAME}/{VOLUME_NAME}'

print(f'Catalog: {CATALOG_NAME}')
print(f'Schema: {SCHEMA_NAME}')
print(f'Volume: {VOLUME_NAME}')
print(f'Volume Path: {VOLUME_PATH}')

## Step 2: Create Catalog

In [0]:
%sql
-- Create catalog if it doesn't exist
CREATE CATALOG IF NOT EXISTS ldp_exercise;

## Step 3: Create Schema

In [0]:
%sql
-- Create schema within the catalog
CREATE SCHEMA IF NOT EXISTS ldp_exercise.exercise_schema;

## Step 4: Create Volume

In [0]:
%sql
-- Create volume within the schema
CREATE VOLUME IF NOT EXISTS ldp_exercise.exercise_schema.raw;

## Step 5: Create Directories in Volume

In [0]:
def create_directory_in_volume(volume_path: str, folder_names: list):
    '''
    Creates multiple directories in the specified volume path using dbutils.fs.
    
    Parameters:
    - volume_path (str): The base volume path
    - folder_names (list): A list of folder names to create
    '''
    print('----------------------------------------------------------------------------------------')
    for folder in folder_names:
        folder_path = f'{volume_path}/{folder}'
        try:
            # Try to list the directory to check if it exists
            dbutils.fs.ls(folder_path)
            print(f'Directory {folder_path} already exists. No action taken.')
        except:
            # Directory doesn't exist, create it
            dbutils.fs.mkdirs(folder_path)
            print(f'Creating folder: {folder_path}')
    print('----------------------------------------------------------------------------------------\n')

# Create folders for orders, customers, products, and payments
create_directory_in_volume(VOLUME_PATH, ['orders', 'customers', 'products', 'payments'])

## Step 6: Delete Existing Files (if resetting)

In [0]:
def delete_source_files(source_path: str):
    """
    Deletes all files in the specified source volume.
    
    Parameters:
    - source_path: The path to the volume containing the files to delete
    """
    import os
    
    print(f'\nSearching for files in {source_path} to delete...')
    try:
        files = dbutils.fs.ls(source_path)
        file_list = [f.path for f in files if not f.isDir()]
        if not file_list:
            print(f'No files found in {source_path}.\n')
        else:
            for file_path in file_list:
                print(f'Deleting file: {file_path}')
                dbutils.fs.rm(file_path)
    except Exception as e:
        print(f'Directory {source_path} does not exist or is empty: {e}')

# Delete existing files if resetting
delete_source_files(f'{VOLUME_PATH}/orders/')
delete_source_files(f'{VOLUME_PATH}/customers/')
delete_source_files(f'{VOLUME_PATH}/products/')
delete_source_files(f'{VOLUME_PATH}/payments/')

## Step 7: Generate Sample Data with Data Quality Issues

This step creates sample JSON files with **intentional data quality issues** that students must resolve:

- **Duplicates**: Some records appear multiple times
- **Missing Values**: NULL values in critical fields
- **Invalid Formats**: Incorrect date formats, invalid email addresses, negative prices
- **Data Type Issues**: Numbers stored as strings, dates as strings in wrong format
- **Inconsistent Data**: Mixed case values, inconsistent state codes
- **Orphaned Records**: References to non-existent entities

In [0]:
def create_sample_data_with_issues():
    """
    Create sample JSON files with intentional data quality issues.
    
    Data includes:
    - Orders: ~500 records with duplicates, missing values, invalid dates
    - Customers: ~200 records with invalid emails, missing addresses, duplicates
    - Products: ~100 records with negative prices, missing categories, invalid SKUs
    - Payments: ~600 records with invalid amounts, missing order references, duplicate transactions
    """
    import json
    import random
    from datetime import datetime, timedelta
    
    print("\n----------------Creating sample JSON files with data quality issues----------------")
    
    # Base date for generating timestamps
    base_date = datetime(2024, 1, 1, 10, 0, 0)
    
    # ========== PRODUCTS DATA ==========
    print("\n📦 Generating products data...")
    categories = ['Electronics', 'Clothing', 'Home & Garden', 'Books', 'Sports', 'Toys', 'Food', 'Beauty']
    product_names = [
        'Laptop Pro 15', 'Wireless Mouse', 'Mechanical Keyboard', 'USB-C Cable',
        'T-Shirt Classic', 'Jeans Denim', 'Sneakers Sport', 'Winter Jacket',
        'Coffee Maker', 'Blender Mix', 'Lamp Desk', 'Plant Pot',
        'Python Guide', 'Data Science Book', 'Novel Fiction', 'Cookbook Recipes',
        'Basketball', 'Yoga Mat', 'Dumbbells Set', 'Running Shoes',
        'Action Figure', 'Board Game', 'Puzzle 1000', 'LEGO Set'
    ]
    
    sample_products = []
    for i in range(100):
        product_id = 1000 + i
        name = random.choice(product_names)
        
        # Issue: Some products have missing category (NULL)
        category = random.choice(categories) if random.random() > 0.15 else None
        
        # Issue: Some prices are negative or zero
        if random.random() < 0.1:
            price = round(random.uniform(-50, 0), 2)  # Negative price
        elif random.random() < 0.05:
            price = 0  # Zero price
        else:
            price = round(random.uniform(10, 500), 2)
        
        # Issue: Some SKUs are invalid (empty or malformed)
        if random.random() < 0.1:
            sku = ""  # Empty SKU
        elif random.random() < 0.05:
            sku = f"INVALID-{product_id}"  # Invalid format
        else:
            sku = f"SKU-{product_id:04d}"
        
        # Issue: Stock quantity sometimes negative
        stock_quantity = random.randint(-10, 100) if random.random() < 0.1 else random.randint(0, 100)
        
        sample_products.append({
            "product_id": product_id,
            "name": name,
            "category": category,
            "price": price,
            "sku": sku,
            "stock_quantity": stock_quantity,
            "created_at": (base_date + timedelta(days=i)).strftime("%Y-%m-%d")
        })
    
    # ========== CUSTOMERS DATA ==========
    print("👥 Generating customers data...")
    first_names = ["John", "Jane", "Bob", "Alice", "Charlie", "Diana", "Eve", "Frank", "Grace", "Henry",
                   "Ivy", "Jack", "Kate", "Liam", "Mary", "Noah", "Olivia", "Paul", "Quinn", "Rachel"]
    last_names = ["Smith", "Johnson", "Williams", "Brown", "Jones", "Garcia", "Miller", "Davis", "Rodriguez", "Martinez"]
    states = ["NY", "CA", "TX", "FL", "IL", "PA", "OH", "GA", "NC", "MI"]
    
    sample_customers = []
    customer_ids_used = set()
    
    for i in range(200):
        customer_id = 2000 + i
        first_name = random.choice(first_names)
        last_name = random.choice(last_names)
        name = f"{first_name} {last_name}"
        
        # Issue: Some emails are invalid or missing
        if random.random() < 0.1:
            email = None  # Missing email
        elif random.random() < 0.1:
            email = f"invalid-email-{customer_id}"  # Invalid email (no @)
        elif random.random() < 0.05:
            email = f"{first_name.lower()}.{last_name.lower()}@"  # Incomplete email
        else:
            email = f"{first_name.lower()}.{last_name.lower()}{i}@example.com"
        
        # Issue: Some addresses are missing
        if random.random() < 0.15:
            address = None
            city = None
            state = None
            zip_code = None
        else:
            address = f"{random.randint(100, 9999)} Main St"
            city = random.choice(["New York", "Los Angeles", "Chicago", "Houston", "Phoenix", "Philadelphia"])
            # Issue: Some states are lowercase or invalid
            if random.random() < 0.1:
                state = random.choice(states).lower()  # Lowercase state
            elif random.random() < 0.05:
                state = "XX"  # Invalid state code
            else:
                state = random.choice(states)
            zip_code = f"{random.randint(10000, 99999)}"
        
        # Issue: Some phone numbers are invalid
        if random.random() < 0.1:
            phone = None
        elif random.random() < 0.1:
            phone = "123"  # Too short
        else:
            phone = f"{random.randint(200, 999)}-{random.randint(100, 999)}-{random.randint(1000, 9999)}"
        
        sample_customers.append({
            "customer_id": customer_id,
            "name": name,
            "email": email,
            "address": address,
            "city": city,
            "state": state,
            "zip_code": zip_code,
            "phone": phone,
            "created_at": (base_date + timedelta(days=i%30)).strftime("%Y-%m-%dT%H:%M:%SZ")
        })
        customer_ids_used.add(customer_id)
    
    # Issue: Add duplicate customers (same customer_id with different data)
    for _ in range(10):
        dup_customer = random.choice(sample_customers).copy()
        dup_customer["created_at"] = (base_date + timedelta(days=random.randint(100, 200))).strftime("%Y-%m-%dT%H:%M:%SZ")
        sample_customers.append(dup_customer)
    
    # ========== ORDERS DATA ==========
    print("🛒 Generating orders data...")
    order_statuses = ['pending', 'processing', 'shipped', 'delivered', 'cancelled', 'returned']
    
    sample_orders = []
    order_ids_used = set()
    
    for i in range(500):
        order_id = 50000 + i
        
        # Issue: Some orders reference non-existent customers
        if random.random() < 0.05:
            customer_id = random.randint(5000, 6000)  # Non-existent customer
        else:
            customer_id = random.choice(list(customer_ids_used))
        
        # Issue: Some timestamps are in wrong format or invalid
        if random.random() < 0.1:
            order_date = (base_date + timedelta(days=random.randint(0, 365), hours=random.randint(0, 23))).strftime("%d/%m/%Y %H:%M")  # Wrong format
        elif random.random() < 0.05:
            order_date = "invalid-date"  # Invalid date
        else:
            order_date = (base_date + timedelta(days=random.randint(0, 365), hours=random.randint(0, 23))).strftime("%Y-%m-%dT%H:%M:%SZ")
        
        # Issue: Some statuses are invalid or missing
        if random.random() < 0.1:
            status = None  # Missing status
        elif random.random() < 0.05:
            status = "INVALID_STATUS"  # Invalid status
        else:
            status = random.choice(order_statuses)
        
        # Issue: Some totals are negative or missing
        if random.random() < 0.1:
            total_amount = round(random.uniform(-100, 0), 2)  # Negative total
        elif random.random() < 0.05:
            total_amount = None  # Missing total
        else:
            total_amount = round(random.uniform(20, 1000), 2)
        
        sample_orders.append({
            "order_id": order_id,
            "customer_id": customer_id,
            "order_date": order_date,
            "status": status,
            "total_amount": total_amount
        })
        order_ids_used.add(order_id)
    
    # Issue: Add duplicate orders
    for _ in range(20):
        dup_order = random.choice(sample_orders).copy()
        sample_orders.append(dup_order)
    
    # ========== PAYMENTS DATA ==========
    print("💳 Generating payments data...")
    payment_methods = ['credit_card', 'debit_card', 'paypal', 'bank_transfer', 'cash']
    payment_statuses = ['pending', 'completed', 'failed', 'refunded']
    
    sample_payments = []
    
    for i in range(600):
        payment_id = 70000 + i
        
        # Issue: Some payments reference non-existent orders
        if random.random() < 0.08:
            order_id = random.randint(60000, 70000)  # Non-existent order
        else:
            order_id = random.choice(list(order_ids_used))
        
        # Issue: Some amounts are negative, zero, or missing
        if random.random() < 0.1:
            amount = round(random.uniform(-200, 0), 2)  # Negative amount
        elif random.random() < 0.05:
            amount = 0  # Zero amount
        elif random.random() < 0.05:
            amount = None  # Missing amount
        else:
            amount = round(random.uniform(10, 1200), 2)
        
        # Issue: Some payment methods are invalid
        if random.random() < 0.1:
            payment_method = None  # Missing method
        elif random.random() < 0.05:
            payment_method = "INVALID_METHOD"  # Invalid method
        else:
            payment_method = random.choice(payment_methods)
        
        # Issue: Some statuses are invalid
        if random.random() < 0.1:
            payment_status = None  # Missing status
        elif random.random() < 0.05:
            payment_status = "UNKNOWN"  # Invalid status
        else:
            payment_status = random.choice(payment_statuses)
        
        # Issue: Some timestamps are in wrong format
        if random.random() < 0.1:
            payment_date = (base_date + timedelta(days=random.randint(0, 365), hours=random.randint(0, 23))).strftime("%m-%d-%Y")  # Wrong format
        else:
            payment_date = (base_date + timedelta(days=random.randint(0, 365), hours=random.randint(0, 23))).strftime("%Y-%m-%dT%H:%M:%SZ")
        
        sample_payments.append({
            "payment_id": payment_id,
            "order_id": order_id,
            "amount": amount,
            "payment_method": payment_method,
            "payment_status": payment_status,
            "payment_date": payment_date
        })
    
    # Issue: Add duplicate payments
    for _ in range(30):
        dup_payment = random.choice(sample_payments).copy()
        sample_payments.append(dup_payment)
    
    # Write files using dbutils.fs.put (newline-delimited JSON)
    try:
        # Products file
        products_file = f'{VOLUME_PATH}/products/00.json'
        products_json_lines = [json.dumps(product) for product in sample_products]
        products_content = '\n'.join(products_json_lines)
        dbutils.fs.put(products_file, products_content, overwrite=True)
        print(f'✅ Created products file: {products_file} ({len(sample_products)} records)')
        
        # Customers file
        customers_file = f'{VOLUME_PATH}/customers/00.json'
        customers_json_lines = [json.dumps(customer) for customer in sample_customers]
        customers_content = '\n'.join(customers_json_lines)
        dbutils.fs.put(customers_file, customers_content, overwrite=True)
        print(f'✅ Created customers file: {customers_file} ({len(sample_customers)} records)')
        
        # Orders file
        orders_file = f'{VOLUME_PATH}/orders/00.json'
        orders_json_lines = [json.dumps(order) for order in sample_orders]
        orders_content = '\n'.join(orders_json_lines)
        dbutils.fs.put(orders_file, orders_content, overwrite=True)
        print(f'✅ Created orders file: {orders_file} ({len(sample_orders)} records)')
        
        # Payments file
        payments_file = f'{VOLUME_PATH}/payments/00.json'
        payments_json_lines = [json.dumps(payment) for payment in sample_payments]
        payments_content = '\n'.join(payments_json_lines)
        dbutils.fs.put(payments_file, payments_content, overwrite=True)
        print(f'✅ Created payments file: {payments_file} ({len(sample_payments)} records)')
        
        return True
    except Exception as e:
        print(f'❌ Error creating sample files: {e}')
        import traceback
        traceback.print_exc()
        return False

# Create sample JSON files with data quality issues
print('\n📝 Creating sample JSON files with intentional data quality issues...')
sample_created = create_sample_data_with_issues()

if sample_created:
    print('\n✅ Successfully created all sample JSON files!')
    print(f'\nFiles created in: {VOLUME_PATH}')
    print('  - products/00.json (~100 products with data issues)')
    print('  - customers/00.json (~210 customers with data issues)')
    print('  - orders/00.json (~520 orders with data issues)')
    print('  - payments/00.json (~630 payments with data issues)')
    print('\n⚠️  Note: These files contain intentional data quality issues that you must resolve!')
else:
    print('\n❌ Could not create sample files automatically.')

## Step 8: Verify Setup

In [0]:
%sql
-- Verify catalog exists
SHOW CATALOGS LIKE 'ldp_exercise';

In [0]:
%sql
-- Verify schema exists
SHOW SCHEMAS IN ldp_exercise LIKE 'exercise_schema';

In [0]:
%sql
-- Verify volume exists
SHOW VOLUMES IN ldp_exercise.exercise_schema LIKE 'raw';

In [0]:
# Verify folders and files
print(f'\nVerifying volume structure:')
print(f'Volume path: {VOLUME_PATH}')
print(f'\nFolders:')
for folder in ['orders', 'customers', 'products', 'payments']:
    folder_path = f'{VOLUME_PATH}/{folder}'
    try:
        files = dbutils.fs.ls(folder_path)
        file_list = [f.name for f in files if not f.isDir()]
        print(f'  {folder}/: {len(file_list)} file(s)')
        if file_list:
            for file in sorted(file_list)[:3]:  # Show first 3 files
                print(f'    - {file}')
    except Exception as e:
        print(f'  {folder}/: NOT FOUND or ERROR - {e}')

## Setup Complete!

Your environment is now configured with:
- Catalog: `ldp_exercise`
- Schema: `exercise_schema`
- Volume: `raw` at `/Volumes/ldp_exercise/exercise_schema/raw`
- Folders: `orders`, `customers`, `products`, `payments`

All tables will be created in the `ldp_exercise.exercise_schema` schema.

**⚠️ Important:** The sample data contains intentional data quality issues that you must identify and resolve in your pipeline.

You can now proceed to the requirements notebook to understand the exercise objectives.